In [ ]:
import gc # gc ---垃圾回收器接口
import pandas as pd
import numpy as np

In [ ]:
LOCAL_QUICK = True
# LOCAL_QUICK = False

sample_persent = 0.1

# MORE_FE = False
MORE_FE = True

FE_V1 = False if MORE_FE else True

In [ ]:
# 文件地址
filepath = 'F:/OneDrive/Learning/DataAnalyticsProject/RePurchaseAnalysis/data_sets/'
# 用户行为日志
user_log = pd.read_csv(filepath + 'user_log_format1.csv')
# 用户画像
user_info = pd.read_csv(filepath + 'user_info_format1.csv')
# 训练数据
train1 = pd.read_csv(filepath + 'train_format1.csv')
# 测试数据
test1 = pd.read_csv(filepath + 'test_format1.csv')

In [ ]:
# # %%time
# # 采样测试
# if LOCAL_QUICK:
#     print('Local quick test, rate is {:.1f}'.format(sample_persent))
#     user_log = user_log.sample(int(len(user_log) * sample_persent))
#     user_info = user_info.sample(int(len(user_info) * sample_persent))
#     train1 = train1.sample(int(len(train1) * sample_persent))
# else:
#     print('All sample train')

for df in [user_log, user_info, train1, test1]:
    print(list(df.columns))
    print(df.shape)

user_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)

In [ ]:
train1['origin'] = 'train'
test1['origin'] = 'test'
matrix = pd.concat([train1, test1], ignore_index=True, sort=True)
matrix.drop(['prob'], axis=1, inplace=True)
matrix = matrix.merge(user_info, on='user_id', how='left')

del df, train1, test1, user_info
gc.collect() # 若被调用时不包含参数，则启动完全的垃圾回收

In [ ]:
# %%time
# 格式化
user_log['user_id'] = user_log['user_id'].astype('int32')
user_log['merchant_id'] = user_log['merchant_id'].astype('int32')
user_log['item_id'] = user_log['item_id'].astype('int32')
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int32')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')

# 缺失值填充
matrix['age_range'].fillna(0, inplace=True)
matrix['gender'].fillna(2, inplace=True)

# # gender 用众数填充（表现更差？）
# matrix['gender'].fillna(matrix['gender'].mode()[0], inplace=True)
# # 年龄用中位数填充
# matrix['age_range'].fillna(matrix['age_range'].median(), inplace=True)

matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')


In [ ]:
matrix = matrix[['origin', 'merchant_id', 'user_id', 'label', 'age_range', 'gender']]
matrix.head()

In [ ]:
## 特征处理
## user特征处理
groups = user_log.groupby(['user_id'])

# 用户交互行为数量 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, how='left', on='user_id')

# 使用agg基于列的聚合操作，统计唯一值个数 item_id, cat_id, merchant_id, brand_id
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, how='left', on='user_id')

temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, how='left', on='user_id')

temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, how='left', on='user_id')

temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, how='left', on='user_id')

# 时间间隔特征 u6 按照小时
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds / 3600
matrix = matrix.merge(temp[['user_id', 'u6']], how='left', on='user_id')

# 统计操作类型为0,1,2,3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, how='left', on='user_id')

del temp
gc.collect()

In [ ]:
matrix.head(10)
matrix.isnull().sum()

In [ ]:
## 商家特征处理
groups = user_log.groupby(['merchant_id'])
# 商家被交互行为的数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, how='left', on='merchant_id')

# 统计商家被交互的user_id,item_id,cat_id,brand_id唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, how='left', on='merchant_id')

# 统计商家被交互action_type唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how = 'left')

del temp
gc.collect()

In [ ]:
matrix.head()

label: 取值集合为{0, 1, -1, NULL}; 取1表示'userid'是'merchantid'的重复买家，取0则反之。取-1表示'user_id'不是给定商家的新客户，因此不在我们预测范围内，但这些记录可能会提供额外信息。测试集这一部分需要预测，因此为NULL。

In [ ]:
# 按照merchant_id统计随机负采样的个数
train2 = pd.read_csv(filepath + 'train_format2.csv')
temp = train2[train2['label'] == -1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, how='left', on='merchant_id')

del train2, temp
gc.collect()

In [ ]:
## 用户特征+商户特征
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'})
matrix = matrix.merge(temp, how='left', on=['user_id', 'merchant_id'])

temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'})
matrix = matrix.merge(temp, how='left', on=['user_id', 'merchant_id'])

temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})
matrix = matrix.merge(temp, how='left', on=['user_id', 'merchant_id'])

temp = groups['time_stamp'].agg([('frist', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['frist']).dt.seconds/3600
temp.drop(['frist', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, how='left', on=['user_id', 'merchant_id'])

del temp, user_log
gc.collect()

In [ ]:
matrix['r1'] = matrix['u9'] / matrix['u7'] # 用户购买点击比
matrix['r2'] = matrix['m8'] / matrix['m6'] # 商户购买点击比
matrix['r3'] = matrix['um7'] / matrix['um5'] # 不同用户不同商户购买点击比

In [ ]:
matrix.fillna(0, inplace=True)


In [ ]:
## 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

del temp
gc.collect()

In [ ]:
## 训练、测试特征集保存
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)

if not LOCAL_QUICK:
    if FE_V1:
        train_data.to_csv('train_data.csv')
        test_data.to_csv('test_data.csv')
    if MORE_FE:
        train_data.to_csv('train_data_moreFE.csv')
        test_data.to_csv('test_data_moreFE.csv')

del matrix
gc.collect()

In [ ]:
del user_log
gc.collect()